In [1]:
import pandas as pd
from pulp import *
PATH = 'data/'

# Read in gameweek data
current_df = pd.read_csv(PATH + '2021-22/gws/merged_gw.csv', index_col=False)
gw_35 = current_df[current_df.GW == 35]

# Create the dictionary 
GW_dict = {}
GWs = current_df.GW.unique()

points_perGW = {}
#for m in range(35):
    
# Solving for the first x weeks, first part before free hit
players = current_df[current_df.GW == 1].name.unique()
for GW in sorted(GWs)[24:25]:
    print(GW)
    player_price_dict = {} # key: player, value: (name, team, position, price, total points)
    print("Number players in week {} is {}".format(GW, len(players)))
    for i, player in enumerate(players):
        player_entries = current_df[current_df.name == player]
        total_points = player_entries[player_entries.GW <= GW]['total_points'].sum()
        player_team = player_entries[player_entries.GW == 1]['team'].values[0]
        player_price = player_entries[player_entries.GW == 1]['value'].values[0]
        player_position = player_entries[player_entries.GW == 1]['position'].values[0]
        player_name = player_entries[player_entries.GW == 1]['name']
        if player not in player_price_dict:
                val = (player,player_team, player_position, player_price, total_points)
                player_price_dict[player] = val
    GW_dict[GW] = player_price_dict
    #print("Player {} had {} price total in all {} played.".format(player, player_price, len(player_entries)))


# # Solving for the best teams from each week x until end, second half!!!!!!!!!!!! done :0
# for GW in sorted(GWs)[25:26]:
#     player_price_dict = {} # key: player, value: (name, team, position, price, total points)
#     players = current_df[current_df.GW == GW].name.unique()
#     # print("Number players in week {} is {}".format(GW, len(players)))
#     for i, player in enumerate(players):
#         player_entries = current_df[current_df.name == player]
#         total_points = player_entries[player_entries.GW >= GW]['total_points'].sum()
#         player_team = player_entries[player_entries.GW == GW]['team'].values[0]
#         player_price = player_entries[player_entries.GW == GW]['value'].values[0]
#         player_position = player_entries[player_entries.GW == GW]['position'].values[0]
#         player_name = player_entries[player_entries.GW == GW]['name']
#         if player not in player_price_dict:
#                 val = (player,player_team, player_position, player_price, total_points)
#                 player_price_dict[player] = val             
#     GW_dict[GW] = player_price_dict
#         #print("Player {} had {} price total in all {} played.".format(player, player_price, len(player_entries)))



gw_35.head()
data = gw_35[['name', 'team', 'position', 'total_points', 'value']]        
data.head()

# Helper variables
POS = data.position.unique()
CLUBS = data.team.unique()

BUDGET = 1000
pos_available = {
'DEF': 5,
'FWD': 3,
'MID': 5,
'GK': 2,
}

# Initialize Variables
solver_dict = {}
points_dict = {}
for GW in GWs[24:25]:
    # print(GW)
    cur_player_price_dict = GW_dict[GW]
    names, teams, positions, prices, points = list(zip(*cur_player_price_dict.values()))
    players = [LpVariable("player_" + str(i), cat="Binary") for i in range(len(names))]

    # Initialize the problem
    prob = LpProblem("FPL_Player_Choices", LpMaximize)
    # Define the objective
    prob += lpSum(players[i] * points[i] for i in range(len(names))) # Objective
    # Build the constraints
    prob += lpSum(players[i] * prices[i] for i in range(len(names))) <= BUDGET # Budget Limit

    for pos in POS:
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) <= pos_available[pos]
    for club in CLUBS:
        prob += lpSum(players[i] for i in range(len(names)) if teams[i] == club) <= 3 # Club Limit

    # Solve the problem
    prob.solve()
    for v in prob.variables():
        if v.varValue != 0:
            index = int(v.name.split("_")[1])
            name = names[index]
            club = teams[index]
            position = positions[index]
            point = points[index]
            price = prices[index]
            # week_point = week_points[index]
            print(name, position, club, point, price, sep=" | ")
            score = str(prob.objective)
            constraint = [str(const) for const in prob.constraints.values()][0]
            solver_dict[name] = name, position, club, point, price, 
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraint = constraint.replace(v.name, str(v.varValue))

    score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
    constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

    # print(value(prob.objective))
    points_dict[GW] = value(prob.objective)


names, positions, clubs, points, prices = list(zip(*solver_dict.values()))
totals_dict = [0]*35
my_team = {}
for i in range(len(names)):
    for GW in sorted(GWs)[:35]:
        player_entries = current_df[current_df.name == names[i]]
        points = player_entries[player_entries.GW == GW]['total_points'].sum()
        totals_dict[GW-1] = points
    my_team[names[i]] = names[i], positions[i], clubs[i], prices[i], totals_dict
    totals_dict = [0]*35
#print(my_team)


#Solve again 4-4-2
# Helper variables
POS = data.position.unique()
CLUBS = data.team.unique()

BUDGET = 1000
pos_available = {
'DEF': [3,5], #3 to 5
'FWD': [1,3], #1 to 3
'MID': [2,5], #2 to 5
'GK': [1,1], #have to have 1 goalie (2)
} # total adds to 11

# Initialize Variables
solver_dict = {}
points_dict = {}
points_total = 0;
for GW in GWs[0:25]:
    names, positions, teams, prices, points = list(zip(*my_team.values()))
    players = [LpVariable("player_" + str(i), cat="Binary") for i in range(len(names))]
    #posi = [LpVariable("pos_" + str(i), cat="Integer") for i in range(4)]

    # Initialie the problem
    prob = LpProblem("FPL_Player_Choices", LpMaximize)
    # Define the objective
    prob += lpSum(players[i] * points[i][GW-1] for i in range(len(names))) # Objective

    #prob += lpSum(posi[i] for i in range(len(names))   ) == 11

    for pos in POS:
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) <= pos_available[pos][1]
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) >= pos_available[pos][0]

    prob += lpSum(players[i] for i in range(len(names))) == 11

    # Solve the problem
    prob.solve()
    captain = 0;
    for v in prob.variables():
        if v.varValue != 0:
            index = int(v.name.split("_")[1])
            name = names[index]
            club = teams[index]
            position = positions[index]
            point = points[index][GW-1]
            if point > captain:
                captain = point
            price = prices[index]
            print(name, position, club, point, price, sep=" | ")
            score = str(prob.objective)
            constraint = [str(const) for const in prob.constraints.values()][0]
            solver_dict[name] = name, position, club, point, price
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraint = constraint.replace(v.name, str(v.varValue))

    score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
    constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

    points_dict[GW] = value(prob.objective) + captain
    points_total += value(prob.objective) + captain
#     points += captain
    print(GW)
    print(value(prob.objective) + captain)
    print("New Total")
    print(points_total)
    points_perGW[GW] = value(prob.objective) + captain
    captain = 0

print("Final")
print(points_total)

print(points_perGW)

25
Number players in week 25 is 553
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/16f452f0d0b8402fa3c8ece47552a539-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/16f452f0d0b8402fa3c8ece47552a539-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3236 RHS
At line 3262 BOUNDS
At line 3816 ENDATA
Problem MODEL has 25 rows, 553 columns and 1659 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1792.86 - 0.00 seconds
Cgl0004I processed model has 25 rows, 437 columns (437 integer (434 of which binary)) and 1311 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 4 int